# PySpark Dataframe Complete Guide (with COVID-19 Dataset)

Spark which is one of the most used tools when it comes to working with Big Data.

While once upon a time Spark used to be heavily reliant on RDD manipulations, Spark has now provided a DataFrame API for us Data Scientists to work with. [Doc](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#).

**Yay~!**

In this notebook, We will learn standard Spark functionalities needed to work with DataFrames, and finally some tips to handle the inevitable errors you will face.

I'm going to skip the Spark Installation part for the sake of the notebook, so please go to [Apache Spark Website](http://spark.apache.org/downloads.html) to install Spark that are right to your work setting.

In [1]:
#we use the findspark library to locate spark on our local machine
import findspark
findspark.init()

In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import time

import pyspark # only run this after findspark.init()
from pyspark.sql import SparkSession, SQLContext
from pyspark.context import SparkContext
from pyspark.sql.functions import * 
from pyspark.sql.types import * 

In [3]:
# Initiate the Spark Session
spark = SparkSession.builder.appName('covid-example').getOrCreate()

24/04/03 16:04:30 WARN Utils: Your hostname, yongjoo-serv-01 resolves to a loopback address: 127.0.1.1; using 172.22.224.80 instead (on interface eno33np0)
24/04/03 16:04:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 16:04:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark

## Data
 We will be working with the Data Science for COVID-19 in South Korea, which is one of the most detailed datasets on the internet for COVID.

Data can be found in this kaggle URL [Link](https://www.kaggle.com/kimjihoo/coronavirusdataset)

### 1. Basic Functions

#### [1] Load (Read) the data

In [5]:
cases = spark.read.load("/data/elastic-notebook/data/covid/Case.csv",
                        format="csv", 
                        sep=",", 
                        inferSchema="true", 
                        header="true")

In [6]:
# First few rows in the file
cases.show()

+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+--------+--------+---------------+-----+--------------------+---------+---------+----------+
| 1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
| 1000002|   Seoul|      Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
| 1000003|   Seoul|        Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
| 1000004|   Seoul|   Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
| 1000005|   Seoul|      Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
| 1000006|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
| 1000007|   Seoul|from other city| true|SMR Newly Planted...|       36|        -|         -|
| 1000008|   Seoul|  Dongdaemun-gu| true|       Dongan Churc

In [7]:
cases.limit(10).toPandas()

,case_id,province,city,group,infection_case,confirmed,latitude,longitude
0,1000001,Seoul,Yongsan-gu,True,Itaewon Clubs,139,37.538621,126.992652
1,1000002,Seoul,Gwanak-gu,True,Richway,119,37.48208,126.901384
2,1000003,Seoul,Guro-gu,True,Guro-gu Call Center,95,37.508163,126.884387
3,1000004,Seoul,Yangcheon-gu,True,Yangcheon Table Tennis Club,43,37.546061,126.874209
4,1000005,Seoul,Dobong-gu,True,Day Care Center,43,37.679422,127.044374
5,1000006,Seoul,Guro-gu,True,Manmin Central Church,41,37.481059,126.894343
6,1000007,Seoul,from other city,True,SMR Newly Planted Churches Group,36,-,-
7,1000008,Seoul,Dongdaemun-gu,True,Dongan Church,17,37.592888,127.056766
8,1000009,Seoul,from other city,True,Coupang Logistics Center,25,-,-
9,1000010,Seoul,Gwanak-gu,True,Wangsung Church,30,37.481735,126.930121


It looks ok right now, but sometimes as we the number of columns increases, the formatting becomes not too great. I have noticed that the following trick helps in displaying in pandas format in my Jupyter Notebook. 

The **.toPandas()** function converts a **Spark Dataframe** into a **Pandas Dataframe**, which is much easier to play with.

#### [2] Change Column Names

To change a single column,

In [8]:
cases = cases.withColumnRenamed("infection_case","infection_source")

To change all columns,

In [9]:
cases = cases.toDF(*['case_id', 'province', 'city', 'group', 'infection_case', 'confirmed',
       'latitude', 'longitude'])

In [10]:
cases.show()

+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|case_id|province|           city|group|      infection_case|confirmed| latitude| longitude|
+-------+--------+---------------+-----+--------------------+---------+---------+----------+
|1000001|   Seoul|     Yongsan-gu| true|       Itaewon Clubs|      139|37.538621|126.992652|
|1000002|   Seoul|      Gwanak-gu| true|             Richway|      119| 37.48208|126.901384|
|1000003|   Seoul|        Guro-gu| true| Guro-gu Call Center|       95|37.508163|126.884387|
|1000004|   Seoul|   Yangcheon-gu| true|Yangcheon Table T...|       43|37.546061|126.874209|
|1000005|   Seoul|      Dobong-gu| true|     Day Care Center|       43|37.679422|127.044374|
|1000006|   Seoul|        Guro-gu| true|Manmin Central Ch...|       41|37.481059|126.894343|
|1000007|   Seoul|from other city| true|SMR Newly Planted...|       36|        -|         -|
|1000008|   Seoul|  Dongdaemun-gu| true|       Dongan Church|       17

#### [3] Change Column Names

We can select a subset of columns using the **select** 

In [11]:
cases = cases.select('province','city','infection_case','confirmed')
cases.show()

+--------+---------------+--------------------+---------+
|province|           city|      infection_case|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

#### [4] Sort by Column

In [12]:
# Simple sort
cases.sort("confirmed").show()

+------------+---------------+--------------------+---------+
|    province|           city|      infection_case|confirmed|
+------------+---------------+--------------------+---------+
|       Seoul|     Gangseo-gu|SJ Investment Cal...|        0|
|  Gangwon-do|              -|contact with patient|        0|
|     Jeju-do|              -|contact with patient|        0|
|       Busan|from other city|Cheongdo Daenam H...|        1|
|       Seoul|     Gangnam-gu|Gangnam Dongin Ch...|        1|
|       Seoul|from other city|Anyang Gunpo Past...|        1|
|       Seoul|from other city|Daejeon door-to-d...|        1|
|       Seoul|              -|         Orange Life|        1|
|      Sejong|              -|                 etc|        1|
|     Gwangju|              -|                 etc|        1|
|      Sejong|from other city|  Shincheonji Church|        1|
|Jeollabuk-do|from other city|  Shincheonji Church|        1|
|Jeollanam-do|from other city|  Shincheonji Church|        1|
|     Je

In [13]:
# Descending Sort
from pyspark.sql import functions as F

cases.sort(F.desc("confirmed")).show()

+-----------------+---------------+--------------------+---------+
|         province|           city|      infection_case|confirmed|
+-----------------+---------------+--------------------+---------+
|            Daegu|         Nam-gu|  Shincheonji Church|     4511|
|            Daegu|              -|contact with patient|      917|
|            Daegu|              -|                 etc|      747|
| Gyeongsangbuk-do|from other city|  Shincheonji Church|      566|
|      Gyeonggi-do|              -|     overseas inflow|      305|
|            Seoul|              -|     overseas inflow|      298|
|            Daegu|   Dalseong-gun|Second Mi-Ju Hosp...|      196|
| Gyeongsangbuk-do|              -|contact with patient|      190|
|            Seoul|              -|contact with patient|      162|
|            Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
| Gyeongsangbuk-do|              -|                 etc|      133|
|            Daegu|         Seo-gu|Hansarang Convale...|      

#### [5] Change Column Type

In [14]:
from pyspark.sql.types import DoubleType, IntegerType, StringType

cases = cases.withColumn('confirmed', F.col('confirmed').cast(IntegerType()))
cases = cases.withColumn('city', F.col('city').cast(StringType()))

cases.show()

+--------+---------------+--------------------+---------+
|province|           city|      infection_case|confirmed|
+--------+---------------+--------------------+---------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|
|   Seoul|      Gwanak-gu|             Richway|      119|
|   Seoul|        Guro-gu| Guro-gu Call Center|       95|
|   Seoul|   Yangcheon-gu|Yangcheon Table T...|       43|
|   Seoul|      Dobong-gu|     Day Care Center|       43|
|   Seoul|        Guro-gu|Manmin Central Ch...|       41|
|   Seoul|from other city|SMR Newly Planted...|       36|
|   Seoul|  Dongdaemun-gu|       Dongan Church|       17|
|   Seoul|from other city|Coupang Logistics...|       25|
|   Seoul|      Gwanak-gu|     Wangsung Church|       30|
|   Seoul|   Eunpyeong-gu|Eunpyeong St. Mar...|       14|
|   Seoul|   Seongdong-gu|    Seongdong-gu APT|       13|
|   Seoul|      Jongno-gu|Jongno Community ...|       10|
|   Seoul|     Gangnam-gu|Samsung Medical C...|        7|
|   Seoul|    

#### [6] Filter 

We can filter a data frame using multiple conditions using AND(&), OR(|) and NOT(~) conditions. For example, we may want to find out all the different infection_case in Daegu with more than 10 confirmed cases.

In [15]:
cases.filter((cases.confirmed>10) & (cases.province=='Daegu')).show()

+--------+------------+--------------------+---------+
|province|        city|      infection_case|confirmed|
+--------+------------+--------------------+---------+
|   Daegu|      Nam-gu|  Shincheonji Church|     4511|
|   Daegu|Dalseong-gun|Second Mi-Ju Hosp...|      196|
|   Daegu|      Seo-gu|Hansarang Convale...|      124|
|   Daegu|Dalseong-gun|Daesil Convalesce...|      101|
|   Daegu|     Dong-gu|     Fatima Hospital|       39|
|   Daegu|           -|     overseas inflow|       41|
|   Daegu|           -|contact with patient|      917|
|   Daegu|           -|                 etc|      747|
+--------+------------+--------------------+---------+



#### [7] GroupBy

In [16]:
from pyspark.sql import functions as F

cases.groupBy(["province","city"]).agg(F.sum("confirmed") ,F.max("confirmed")).show()

+----------------+---------------+--------------+--------------+
|        province|           city|sum(confirmed)|max(confirmed)|
+----------------+---------------+--------------+--------------+
|Gyeongsangnam-do|       Jinju-si|             9|             9|
|           Seoul|        Guro-gu|           139|            95|
|           Seoul|     Gangnam-gu|            18|             7|
|         Daejeon|              -|           100|            55|
|    Jeollabuk-do|from other city|             6|             3|
|Gyeongsangnam-do|Changnyeong-gun|             7|             7|
|           Seoul|              -|           561|           298|
|         Jeju-do|from other city|             1|             1|
|Gyeongsangbuk-do|              -|           345|           190|
|Gyeongsangnam-do|   Geochang-gun|            18|            10|
|Gyeongsangbuk-do|        Gumi-si|            10|            10|
|         Incheon|from other city|           117|            53|
|           Busan|       

Or if we don’t like the new column names, we can use the **alias** keyword to rename columns in the agg command itself.

In [17]:
cases.groupBy(["province","city"]).agg(
    F.sum("confirmed").alias("TotalConfirmed"),\
    F.max("confirmed").alias("MaxFromOneConfirmedCase")\
    ).show()

+----------------+---------------+--------------+-----------------------+
|        province|           city|TotalConfirmed|MaxFromOneConfirmedCase|
+----------------+---------------+--------------+-----------------------+
|Gyeongsangnam-do|       Jinju-si|             9|                      9|
|           Seoul|        Guro-gu|           139|                     95|
|           Seoul|     Gangnam-gu|            18|                      7|
|         Daejeon|              -|           100|                     55|
|    Jeollabuk-do|from other city|             6|                      3|
|Gyeongsangnam-do|Changnyeong-gun|             7|                      7|
|           Seoul|              -|           561|                    298|
|         Jeju-do|from other city|             1|                      1|
|Gyeongsangbuk-do|              -|           345|                    190|
|Gyeongsangnam-do|   Geochang-gun|            18|                     10|
|Gyeongsangbuk-do|        Gumi-si|    

#### [8] Joins

Here, We will go with the region file which contains region information such as elementary_school_count, elderly_population_ratio, etc.

In [18]:
regions = spark.read.load("/data/elastic-notebook/data/covid/Region.csv",
                          format="csv", 
                          sep=",", 
                          inferSchema="true", 
                          header="true")

regions.limit(10).toPandas()

,code,province,city,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,10000,Seoul,Seoul,37.566953,126.977977,607,830,48,1.44,15.38,5.8,22739
1,10010,Seoul,Gangnam-gu,37.518421,127.047222,33,38,0,4.18,13.17,4.3,3088
2,10020,Seoul,Gangdong-gu,37.530492,127.123837,27,32,0,1.54,14.55,5.4,1023
3,10030,Seoul,Gangbuk-gu,37.639938,127.025508,14,21,0,0.67,19.49,8.5,628
4,10040,Seoul,Gangseo-gu,37.551166,126.849506,36,56,1,1.17,14.39,5.7,1080
5,10050,Seoul,Gwanak-gu,37.478290,126.951502,22,33,1,0.89,15.12,4.9,909
6,10060,Seoul,Gwangjin-gu,37.538712,127.082366,22,33,3,1.16,13.75,4.8,723
7,10070,Seoul,Guro-gu,37.495632,126.887650,26,34,3,1.00,16.21,5.7,741
8,10080,Seoul,Geumcheon-gu,37.456852,126.895229,18,19,0,0.96,16.15,6.7,475
9,10090,Seoul,Nowon-gu,37.654259,127.056294,42,66,6,1.39,15.40,7.4,952


In [19]:
# Left Join 'Case' with 'Region' on Province and City column
cases = cases.join(regions, ['province','city'],how='left')
cases.limit(10).toPandas()

,province,city,infection_case,confirmed,code,latitude,longitude,elementary_school_count,kindergarten_count,university_count,academy_ratio,elderly_population_ratio,elderly_alone_ratio,nursing_home_count
0,Seoul,Yongsan-gu,Itaewon Clubs,139,10210.0,37.532768,126.990021,15.0,13.0,1.0,0.68,16.87,6.5,435.0
1,Seoul,Gwanak-gu,Richway,119,10050.0,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0
2,Seoul,Guro-gu,Guro-gu Call Center,95,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
3,Seoul,Yangcheon-gu,Yangcheon Table Tennis Club,43,10190.0,37.517189,126.866618,30.0,43.0,0.0,2.26,13.55,5.5,816.0
4,Seoul,Dobong-gu,Day Care Center,43,10100.0,37.668952,127.047082,23.0,26.0,1.0,0.95,17.89,7.2,485.0
5,Seoul,Guro-gu,Manmin Central Church,41,10070.0,37.495632,126.887650,26.0,34.0,3.0,1.00,16.21,5.7,741.0
6,Seoul,from other city,SMR Newly Planted Churches Group,36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Seoul,Dongdaemun-gu,Dongan Church,17,10110.0,37.574552,127.039721,21.0,31.0,4.0,1.06,17.26,6.7,832.0
8,Seoul,from other city,Coupang Logistics Center,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Seoul,Gwanak-gu,Wangsung Church,30,10050.0,37.478290,126.951502,22.0,33.0,1.0,0.89,15.12,4.9,909.0


### 2. Use SQL with DataFrames

We first register the cases dataframe to a temporary table cases_table on which we can run SQL operations. As you can see, the result of the SQL select statement is again a Spark Dataframe.

All complex SQL queries like GROUP BY, HAVING, AND ORDER BY clauses can be applied in 'Sql' function

In [20]:
cases.registerTempTable('cases_table')
newDF = spark.sql('select * from cases_table where confirmed > 100')
newDF.show()

/home/zl20/kishu-venv/lib/python3.8/site-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


+-----------------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|         province|           city|      infection_case|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|
+-----------------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+
|            Seoul|     Yongsan-gu|       Itaewon Clubs|      139|10210|37.532768|126.990021|                     15|                13|               1|         0.68|                   16.87|                6.5|               435|
|            Seoul|      Gwanak-gu|             Richway|      119|10050|

### 3. Create New Columns

There are many ways that you can use to create a column in a PySpark Dataframe.

#### [1] Using Spark Native Functions

We can use .withcolumn along with PySpark SQL functions to create a new column. In essence, you can find String functions, Date functions, and Math functions already implemented using Spark functions. Our first function, the F.col function gives us access to the column. So if we wanted to add 100 to a column, we could use F.col as:

In [21]:
import pyspark.sql.functions as F

casesWithNewConfirmed = cases.withColumn("NewConfirmed", 100 + F.col("confirmed"))
casesWithNewConfirmed.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+------------+
|province|           city|      infection_case|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|NewConfirmed|
+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+------------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|10210|37.532768|126.990021|                     15|                13|               1|         0.68|                   16.87|                6.5|               435|         239|
|   Seoul|      Gwanak-gu|             Richway|      119

We can also use math functions like F.exp function:

In [22]:
casesWithExpConfirmed = cases.withColumn("ExpConfirmed", F.exp("confirmed"))
casesWithExpConfirmed.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------+
|province|           city|      infection_case|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|        ExpConfirmed|
+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+--------------------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|10210|37.532768|126.990021|                     15|                13|               1|         0.68|                   16.87|                6.5|               435|2.327732040478862E60|
|   Seoul|      Gwanak-g

#### [2] Using Spark UDFs

Sometimes we want to do complicated things to a column or multiple columns. This could be thought of as a map operation on a PySpark Dataframe to a single column or multiple columns. While Spark SQL functions do solve many use cases when it comes to column creation, I use Spark UDF whenever I need more matured Python functionality. \

To use Spark UDFs, we need to use the F.udf function to convert a regular python function to a Spark UDF. We also need to specify the return type of the function. In this example the return type is StringType()

In [23]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

def casesHighLow(confirmed):
    if confirmed < 50: 
        return 'low'
    else:
        return 'high'
    
#convert to a UDF Function by passing in the function and return type of function
casesHighLowUDF = F.udf(casesHighLow, StringType())
CasesWithHighLow = cases.withColumn("HighLow", casesHighLowUDF("confirmed"))
CasesWithHighLow.show()

+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-------+
|province|           city|      infection_case|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|HighLow|
+--------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+-------+
|   Seoul|     Yongsan-gu|       Itaewon Clubs|      139|10210|37.532768|126.990021|                     15|                13|               1|         0.68|                   16.87|                6.5|               435|   high|
|   Seoul|      Gwanak-gu|             Richway|      119|10050| 37.47829|126

#### [3] Using Pandas UDF

This allows you to use pandas functionality with Spark. I generally use it when I have to run a groupBy operation on a Spark dataframe or whenever I need to create rolling features
 
The way we use it is by using the F.pandas_udf decorator. **We assume here that the input to the function will be a pandas data frame**

The only complexity here is that we have to provide a schema for the output Dataframe. We can use the original schema of a dataframe to create the outSchema.

In [24]:
cases.printSchema()

root
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- code: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- elementary_school_count: integer (nullable = true)
 |-- kindergarten_count: integer (nullable = true)
 |-- university_count: integer (nullable = true)
 |-- academy_ratio: double (nullable = true)
 |-- elderly_population_ratio: double (nullable = true)
 |-- elderly_alone_ratio: double (nullable = true)
 |-- nursing_home_count: integer (nullable = true)



In [25]:
from pyspark.sql.types import IntegerType, StringType, DoubleType, BooleanType
from pyspark.sql.types import StructType, StructField

# Declare the schema for the output of our function

outSchema = StructType([StructField('province',StringType(),True),
                        StructField('city',StringType(),True),
                        StructField('infection_case',StringType(),True),
                        StructField('confirmed',IntegerType(),True),
                        StructField('latitude',DoubleType(),True),
                        StructField('longitude',DoubleType(),True),
                        StructField('normalized_confirmed',DoubleType(),True),
                        StructField('academy_ratio',DoubleType(),True),
                        StructField('code',IntegerType(),True),
                        StructField('elderly_alone_ratio',DoubleType(),True),
                        StructField('elderly_population_ratio',DoubleType(),True),
                        StructField('elementary_school_count',IntegerType(),True),
                        StructField('kindergarten_count',IntegerType(),True),
                        StructField('nursing_home_count',IntegerType(),True),
                        StructField('university_count',IntegerType(),True)
                       ])

In [26]:
# decorate our function with pandas_udf decorator
@F.pandas_udf(outSchema, F.PandasUDFType.GROUPED_MAP)
def subtract_mean(pdf):
    # pdf is a pandas.DataFrame
    v = pdf.confirmed
    v = v - v.mean()
    pdf['normalized_confirmed'] = v
    return pdf

confirmed_groupwise_normalization = cases.groupby("infection_case").apply(subtract_mean)

confirmed_groupwise_normalization.limit(10).toPandas()

/home/zl20/kishu-venv/lib/python3.8/site-packages/pyspark/sql/pandas/group_ops.py:104: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


,province,city,infection_case,confirmed,latitude,longitude,normalized_confirmed,academy_ratio,code,elderly_alone_ratio,elderly_population_ratio,elementary_school_count,kindergarten_count,nursing_home_count,university_count
0,Seoul,from other city,Anyang Gunpo Pastors Group,1,NaN,NaN,-10.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Gyeonggi-do,Anyang-si,Anyang Gunpo Pastors Group,22,37.394258,126.956752,10.500000,1.86,20170.0,5.1,12.88,41.0,79.0,1099.0,4.0
2,Seoul,Yangcheon-gu,Biblical Language study meeting,3,37.517189,126.866618,0.000000,2.26,10190.0,5.5,13.55,30.0,43.0,816.0,0.0
3,Gyeongsangbuk-do,Bonghwa-gun,Bonghwa Pureun Nursing Home,68,36.893099,128.732568,0.000000,0.37,60080.0,20.0,35.26,14.0,17.0,47.0,0.0
4,Gyeonggi-do,Seongnam-si,Bundang Jesaeng Hospital,22,37.420000,127.126703,0.000000,2.08,20120.0,5.6,13.52,72.0,127.0,2095.0,3.0
5,Gyeongsangnam-do,Changnyeong-gun,Changnyeong Coin Karaoke,7,35.544603,128.492330,0.000000,0.80,61120.0,18.4,29.80,17.0,20.0,129.0,0.0
6,Busan,from other city,Cheongdo Daenam Hospital,1,NaN,NaN,-39.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Daegu,from other city,Cheongdo Daenam Hospital,2,NaN,NaN,-38.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Gyeongsangbuk-do,Cheongdo-gun,Cheongdo Daenam Hospital,119,35.647361,128.734382,78.333333,0.63,60200.0,21.0,36.55,11.0,14.0,85.0,0.0
9,Seoul,from other city,Coupang Logistics Center,25,NaN,NaN,-19.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 4. Spark Window Functions

We will simply look at some of the most important and useful window functions available.

In [27]:
timeprovince = spark.read.load("/data/elastic-notebook/data/covid/TimeProvince.csv",
                          format="csv", 
                          sep=",", 
                          inferSchema="true", 
                          header="true")

timeprovince.show()

+----------+----+-----------------+---------+--------+--------+
|      date|time|         province|confirmed|released|deceased|
+----------+----+-----------------+---------+--------+--------+
|2020-01-20|  16|            Seoul|        0|       0|       0|
|2020-01-20|  16|            Busan|        0|       0|       0|
|2020-01-20|  16|            Daegu|        0|       0|       0|
|2020-01-20|  16|          Incheon|        1|       0|       0|
|2020-01-20|  16|          Gwangju|        0|       0|       0|
|2020-01-20|  16|          Daejeon|        0|       0|       0|
|2020-01-20|  16|            Ulsan|        0|       0|       0|
|2020-01-20|  16|           Sejong|        0|       0|       0|
|2020-01-20|  16|      Gyeonggi-do|        0|       0|       0|
|2020-01-20|  16|       Gangwon-do|        0|       0|       0|
|2020-01-20|  16|Chungcheongbuk-do|        0|       0|       0|
|2020-01-20|  16|Chungcheongnam-do|        0|       0|       0|
|2020-01-20|  16|     Jeollabuk-do|     

#### [1] Ranking

You can get rank as well as dense_rank on a group using this function. For example, you may want to have a column in your cases table that provides the rank of infection_case based on the number of infection_case in a province. We can do this by:

In [28]:
from pyspark.sql.window import Window
windowSpec = Window().partitionBy(['province']).orderBy(F.desc('confirmed'))
cases.withColumn("rank",F.rank().over(windowSpec)).show()

+-----------------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+----+
|         province|           city|      infection_case|confirmed| code| latitude| longitude|elementary_school_count|kindergarten_count|university_count|academy_ratio|elderly_population_ratio|elderly_alone_ratio|nursing_home_count|rank|
+-----------------+---------------+--------------------+---------+-----+---------+----------+-----------------------+------------------+----------------+-------------+------------------------+-------------------+------------------+----+
|            Busan|     Dongnae-gu|       Onchun Church|       39|11060| 35.20506|129.083673|                     22|                31|               0|         1.98|                   17.53|                7.7|               608|   1|
|            Busan|              -|     overseas inf

#### [2] Lag Variables

Sometimes our data science models may need **lag based** features. For example, a model might have variables like the price last week or sales quantity the previous day. We can create such features using the lag function with window functions. \

Here I am trying to get the confirmed cases 7 days before. I am filtering to show the results as the first few days of corona cases were zeros. You can see here that the lag_7 day feature is shifted by 7 days.

In [29]:
from pyspark.sql.window import Window

windowSpec = Window().partitionBy(['province']).orderBy('date')

timeprovinceWithLag = timeprovince.withColumn("lag_7",F.lag("confirmed", 7).over(windowSpec))

timeprovinceWithLag.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+-----+
|      date|time|province|confirmed|released|deceased|lag_7|
+----------+----+--------+---------+--------+--------+-----+
|2020-03-11|   0|   Busan|       98|      21|       0|   92|
|2020-03-12|   0|   Busan|       99|      29|       0|   92|
|2020-03-13|   0|   Busan|      100|      36|       0|   95|
|2020-03-14|   0|   Busan|      103|      40|       0|   96|
|2020-03-15|   0|   Busan|      106|      52|       1|   96|
|2020-03-16|   0|   Busan|      107|      53|       1|   96|
|2020-03-17|   0|   Busan|      107|      54|       1|   96|
|2020-03-18|   0|   Busan|      107|      58|       1|   98|
|2020-03-19|   0|   Busan|      107|      58|       1|   99|
|2020-03-20|   0|   Busan|      108|      60|       1|  100|
|2020-03-21|   0|   Busan|      108|      67|       1|  103|
|2020-03-22|   0|   Busan|      108|      69|       1|  106|
|2020-03-23|   0|   Busan|      109|      71|       1|  107|
|2020-03-24|   0|   Busa

#### [3] Rolling Aggregations

For example, we might want to have a rolling 7-day sales sum/mean as a feature for our sales regression model. Let us calculate the rolling mean of confirmed cases for the last 7 days here. This is what a lot of the people are already doing with this dataset to see the real trends.

In [30]:
from pyspark.sql.window import Window

# we only look at the past 7 days in a particular window including the current_day. 
# Here 0 specifies the current_row and -6 specifies the seventh row previous to current_row. 
# Remember we count starting from 0.

# If we had used rowsBetween(-7,-1), we would just have looked at past 7 days of data and not the current_day
windowSpec = Window().partitionBy(['province']).orderBy('date').rowsBetween(-6,0)

timeprovinceWithRoll = timeprovince.withColumn("roll_7_confirmed",F.mean("confirmed").over(windowSpec))

timeprovinceWithRoll.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+------------------+
|      date|time|province|confirmed|released|deceased|  roll_7_confirmed|
+----------+----+--------+---------+--------+--------+------------------+
|2020-03-11|   0|   Busan|       98|      21|       0| 95.57142857142857|
|2020-03-12|   0|   Busan|       99|      29|       0| 96.57142857142857|
|2020-03-13|   0|   Busan|      100|      36|       0| 97.28571428571429|
|2020-03-14|   0|   Busan|      103|      40|       0| 98.28571428571429|
|2020-03-15|   0|   Busan|      106|      52|       1| 99.71428571428571|
|2020-03-16|   0|   Busan|      107|      53|       1|101.28571428571429|
|2020-03-17|   0|   Busan|      107|      54|       1|102.85714285714286|
|2020-03-18|   0|   Busan|      107|      58|       1|104.14285714285714|
|2020-03-19|   0|   Busan|      107|      58|       1|105.28571428571429|
|2020-03-20|   0|   Busan|      108|      60|       1|106.42857142857143|
|2020-03-21|   0|   Busan|      108|  

One could also find a use for **rowsBetween(Window.unboundedPreceding, Window.currentRow)** function, where we take the rows between the first row in a window and the current_row to get running totals. I am calculating cumulative_confirmed here.

In [31]:
from pyspark.sql.window import Window

windowSpec = Window().partitionBy(['province']).orderBy('date').rowsBetween(Window.unboundedPreceding,Window.currentRow)

timeprovinceWithRoll = timeprovince.withColumn("cumulative_confirmed",F.sum("confirmed").over(windowSpec))

timeprovinceWithRoll.filter(timeprovinceWithLag.date>'2020-03-10').show()

+----------+----+--------+---------+--------+--------+--------------------+
|      date|time|province|confirmed|released|deceased|cumulative_confirmed|
+----------+----+--------+---------+--------+--------+--------------------+
|2020-03-11|   0|   Busan|       98|      21|       0|                1408|
|2020-03-12|   0|   Busan|       99|      29|       0|                1507|
|2020-03-13|   0|   Busan|      100|      36|       0|                1607|
|2020-03-14|   0|   Busan|      103|      40|       0|                1710|
|2020-03-15|   0|   Busan|      106|      52|       1|                1816|
|2020-03-16|   0|   Busan|      107|      53|       1|                1923|
|2020-03-17|   0|   Busan|      107|      54|       1|                2030|
|2020-03-18|   0|   Busan|      107|      58|       1|                2137|
|2020-03-19|   0|   Busan|      107|      58|       1|                2244|
|2020-03-20|   0|   Busan|      108|      60|       1|                2352|
|2020-03-21|

### 5. Pivot DataFrames

Sometimes we may need to have the dataframe in flat format. This happens frequently in movie data where we may want to show genres as columns instead of rows. We can use pivot to do this. Here I am trying to get one row for each date and getting the province names as columns.

In [32]:
pivotedTimeprovince = timeprovince.groupBy('date').pivot('province') \
.agg(F.sum('confirmed').alias('confirmed') , F.sum('released').alias('released'))

pivotedTimeprovince.limit(10).toPandas()

24/04/03 16:04:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,date,Busan_confirmed,Busan_released,Chungcheongbuk-do_confirmed,Chungcheongbuk-do_released,Chungcheongnam-do_confirmed,Chungcheongnam-do_released,Daegu_confirmed,Daegu_released,Daejeon_confirmed,...,Jeollabuk-do_confirmed,Jeollabuk-do_released,Jeollanam-do_confirmed,Jeollanam-do_released,Sejong_confirmed,Sejong_released,Seoul_confirmed,Seoul_released,Ulsan_confirmed,Ulsan_released
0,2020-01-21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2020-04-30,137,116,45,41,143,127,6852,6144,40,...,18,11,15,11,46,38,633,453,43,37
2,2020-03-13,100,36,27,4,115,7,5928,251,22,...,7,4,4,1,32,0,225,40,27,3
3,2020-03-07,96,2,20,1,92,0,5084,18,18,...,7,2,4,0,2,0,108,27,23,1
4,2020-02-15,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,14,2,0,0
5,2020-02-04,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,5,0,0,0
6,2020-05-23,144,131,59,45,145,141,6873,6516,45,...,21,19,18,16,47,47,762,604,50,42
7,2020-02-12,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,14,2,0,0
8,2020-05-08,140,123,47,41,143,130,6859,6277,41,...,19,14,16,12,46,45,637,520,44,37
9,2020-05-24,144,131,59,46,145,141,6873,6518,45,...,21,19,18,17,47,47,768,604,50,42


### 6. Other Opertions

#### [1] Caching

Spark works on the lazy execution principle. What that means is that nothing really gets executed until you use an action function like the .count() on a dataframe. And if you do a .count function, it generally helps to cache at this step. So I have made it a point to cache() my dataframes whenever I do a .count() operation.

In [33]:
cases.cache().count()

174

#### [2] Save and Load from an intermediate step

When you work with Spark you will frequently run with memory and storage issues. While in some cases such issues might be resolved using techniques like broadcasting, salting or cache, sometimes just interrupting the workflow and saving and reloading the whole dataframe at a crucial step has helped me a lot. This helps spark to let go of a lot of memory that gets utilized for storing intermediate shuffle data and unused caches.

In [34]:
cases.write.mode("overwrite").parquet("/data/elastic-notebook/tmp/cases.parquet")
cases.unpersist()
spark.read.load("/data/elastic-notebook/tmp/cases.parquet")

DataFrame[province: string, city: string, infection_case: string, confirmed: int, code: int, latitude: double, longitude: double, elementary_school_count: int, kindergarten_count: int, university_count: int, academy_ratio: double, elderly_population_ratio: double, elderly_alone_ratio: double, nursing_home_count: int]

#### [3] Repartitioning

You might want to repartition your data if you feel your data has been skewed while working with all the transformations and joins. The simplest way to do it is by using:

In [35]:
cases = cases.repartition(1000)

Sometimes you might also want to repartition by a known scheme as this scheme might be used by a certain join or aggregation operation later on. You can use multiple columns to repartition using:

In [36]:
cases = cases.repartition('code')

Then, we can get the number of partitions in a data frame using:

In [37]:
cases.rdd.getNumPartitions()

1

You can also check out the distribution of records in a partition by using the glom function. This helps in understanding the skew in the data that happens while working with various transformations.

#### [4] Reading Parquet File in Local
Sometimes you might want to read the parquet files in a system where Spark is not available. In such cases, I normally use the below code:

In [38]:
from glob import glob
def load_df_from_parquet(parquet_directory):
    df = pd.DataFrame()
    for file in glob(f"{parquet_directory}/*"):
        df = pd.concat([df,pd.read_parquet(file)])
    return df

### 7. Close Spark Instance

In [39]:
spark.stop()

Reference:
>https://medium.com/@rahul_agarwal